# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [44]:
!cat publications.tsv

pub_date	title	venue	excerpt	citation	url_slug	paper_url
2023-07-14	Neural Network-Based Self-tuning Kinematic Control and Dynamic Compensation for Mobile Robots	 Proceedings of 10th International Conference on Mechatronics and Control Engineering 	This paper proposes a neural network-based self-tuning kinematic controller and dynamic compensation for tracking trajectories, which applied, e.g., in cases where mobile robots are subject to: continuous parametric changes; different trajectories and external disturbances where online gain tuning is a desirable choice. For this kind of controller, the kinematic and dynamic model was developed considering that the mobile robot is confirmed by differential platform, and the operating point is not located at the center of wheel’s axes. The artificial neural network estimates the states of the mobile robot while the gradient descent optimization algorithm adjusts the controller gains that attain the smaller position tracking error, the dynamic 

## Import pandas

We are using the very handy pandas library for dataframes.

In [30]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [45]:
publications = pd.read_csv("publications.tsv", sep="\t", header=0)
publications


,pub_date,title,venue,excerpt,citation,url_slug,paper_url
0,2023-07-14,Neural Network-Based Self-tuning Kinematic Con...,Proceedings of 10th International Conference ...,This paper proposes a neural network-based sel...,"Recalde, L.F., Guevara, B.S., Zea, D.J., Andal...",paper-title-number-1,https://link.springer.com/chapter/10.1007/978-...
1,2023-02-20,A Comparative Study between NMPC and Baseline ...,"Conceptual Design, Modeling, and Control Strat...","Transport, rescue, search, surveillance, and d...","Guevara, B.S.; Recalde, L.F.; Varela-Aldás, J....",Paper-title-number-2,https://www.mdpi.com/2504-446X/7/2/144
2,2022-11-01,Constrained Visual Servoing of Quadrotors Base...,IFAC-PapersOnLine,One of the main issues of visual servoing sche...,"Luis F. Recalde, Jose Varela, Bryan S. Guevara...",Paper-title-number-3,https://www.sciencedirect.com/science/article/...
3,2022-08-30,Non Immersive Virtual Laboratory Applied to Ro...,Advances and Trends in Artificial Intelligenc...,This article presents a non immersive virtual ...,"Bastidas, D.A., Recalde, L.F., Constante, P.N....",Paper-title-number-4,https://link.springer.com/chapter/10.1007/978-...
4,2022-06-22,System Identification and Nonlinear Model Pred...,Robotic Systems and Automatic Control: Mathema...,Accurate trajectory tracking is a critical pro...,"Recalde, L.F.; Guevara, B.S.; Carvajal, C.P.; ...",Paper-title-number-5,https://www.mdpi.com/1424-8220/22/13/4712
5,2022-07-26,Nonlinear MPC for Multiple Quadrotors in Dynam...,International Conference on Unmanned Aircraft ...,Accurate trajectory tracking for multiple quad...,"L. F. Recalde, B. S. Guevara, V. Andaluz, J. V...",Paper-title-number-6,https://ieeexplore.ieee.org/document/9836150
6,2020-12-19,Dynamic Simulation and Kinematic Control for A...,Advances in Emerging Trends and Technologies,Dynamic Simulation and Kinematic Control for ...,"Zea, D.J., Guevara, B.S., Recalde, L.F., Andal...",Paper-title-number-7,https://link.springer.com/chapter/10.1007/978-...
7,2021-07-30,Optimal control and dynamic compensation of a ...,The 2nd International Conference on Distribut...,This work proposes a controller for tracking t...,"B. S. Guevara, L. F. Recalde, D. J. Zea and V....",Paper-title-number-8,https://ieeexplore.ieee.org/document/9740094
8,2021-04-27,Three-Dimensional Unified Motion Control of a ...,"Sensors, Devices and Advanced Applications fo...",Technological advances in recent years have sh...,"Ortiz, J.S.; Palacios-Navarro, G.; Andaluz, V....",Paper-title-number-9,https://www.mdpi.com/1424-8220/21/9/3057
9,2020-09-04,Optimal Control Problem of a Differential Driv...,Trends in Artificial Intelligence Theory and ...,This paper proposes an optimal control to trac...,"Recalde, L.F., Guevara, B.S., Cuzco, G., Andal...",Paper-title-number-10,https://link.springer.com/chapter/10.1007/978-...


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [46]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [47]:
import os
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"
    
    if len(str(item.paper_url)) > 5:
        md += "\n[Download paper here](" + item.paper_url + ")\n" 
        
    md += "\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [43]:
!ls ../_publications/

2022-11-01-Paper-title-number-3.md  2023-07-14-paper-title-number-1.md
2023-02-20-Paper-title-number-2.md


In [48]:
!cat ../_publications/2023-02-20-Paper-title-number-2.md

---
title: "A Comparative Study between NMPC and Baseline Feedback Controllers for UAV Trajectory Tracking"
collection: publications
permalink: /publication/2023-02-20-Paper-title-number-2
excerpt: 'Transport, rescue, search, surveillance, and disaster relief tasks are some applications that can be developed with unmanned aerial vehicles (UAVs), where accurate trajectory tracking is a crucial property to operate in a cluttered environment or under uncertainties. However, this is challenging due to high nonlinear dynamics, system constraints, and uncertainties presented in cluttered environments. Hence, uncertainties in the form of unmodeled dynamics, aerodynamic effects, and external disturbances such as wind can produce unstable feedback control schemes, introducing significant positional tracking errors. This work presents a detailed comparative study between controllers such as nonlinear model predictive control (NMPC) and non-predictive baseline feedback controllers, with particula